In [1]:
from pytrends.request import TrendReq
import pandas as pd
import os
import os.path
import pandas as pd
import time
import psycopg2 
import psycopg2.extras as extras
import matplotlib.pyplot as plt

In [ ]:
def menu():
    print(30 * "-", "MENU", 30 * "-")
    print("1. Lấy dữ liệu trending từ file. ")
    print("2. Xuất báo cáo top 10 trending")
    print("3. Xuất báo cáo search keyword in 2020")
    print("4. Vẽ biểu đồ line chart top 5 trending các từ khóa tìm kiếm nhiều nhất 2020")
    print("5. Vẽ biểu đồ bar chart top 5 trending các từ khóa tìm kiếm nhiều nhất 2019")
    print("6. Thống kê tìm kiếm top trending 5 từ khóa trong 2 năm 2020, 2019")
    print(9 * ".")
    print("99.Thoát")

#loop=True
#while loop:
#    menu()
#    choice=input("Enter join choice[1-5]:")
#    if choice=='1':
#        print("Menu 1 has been selected")
#    elif choice=='99':
#        loop=False
#        menu()
#    elif choice=='5':
#        loop=False
menu()
choice=input("Enter join choice[1-5]:")
if choice=='1':
    fromDate=input('Tu ngay')
    to_date=input('Den Ngay')
elif choice=='99':
    menu()
elif choice=='5':
    print('Exit')

In [2]:
def checkExitsFile(inputFile):
    #if os.path.isfile("/{file}.{ext}".format(file=inputFile, ext="xlsx")):
    if os.path.isfile(inputFile):
        return True
    else:
        return False



In [10]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn, cur = None, None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(
            host="localhost", port="5432",
            database="googleTrends",
            user="postgres",
            password="Qu@ng@nh")
        # create a cursor
        cur = conn.cursor()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error while excuting SQL" + error)

    return conn, cur

In [12]:
def getDataFromGoogleTrend(inputFile,FromDate,ToDate):
    
    strTimeFrame=FromDate+' '+ToDate
    checkFile=checkExitsFile(inputFile)
    if checkFile==True:
        df_raw = pd.read_excel(inputFile, sheet_name='key_word')
        #Thay the cot co ky tu / thanh _
        df = df_raw.rename(columns=lambda b: b.replace("/ ","_"))
        column_list = df.columns.ravel()
        
        #lay du lieu tu googletrends
        df_db=[]
        pytrend = TrendReq(hl='vi-VN', tz=360, timeout=10)
        for i in column_list:
            dataset = []
            keywords = list(df[i].dropna()) 
            for x in range(0,len(keywords)):
                keyword =[keywords[x]]
                pytrend.build_payload(
                    kw_list=keyword,
                    cat=0,
                    timeframe=strTimeFrame,
                    geo='VN',
                    gprop=''
                )
                data=pytrend.interest_over_time()
                if not data.empty:
                    data = data.drop(labels=['isPartial'],axis='columns')
                    dataset.append(data)
            result=pd.concat(dataset,axis=1)

           #Tao dataframe cau truc giong voi DB 
            result.reset_index(level=0, inplace=True)
            df_melt=pd.melt(result, id_vars=["date"],var_name="Keywords", value_name="Value")
            df_melt['Trend_type'] = i   
            #result_final=pd.concat([df_melt],ignore_index=True)
            df_db.append(df_melt)
        df_db_final = pd.concat(df_db)        
     
        return df_db_final
        
    else:
        print('File khong ton tai')
    

        
            

In [13]:
df=getDataFromGoogleTrend('keytrends.xlsx','2019-01-01','2020-12-31')

In [14]:
print(df)

           date              Keywords  Value Trend_type
0    2019-01-06  Bầu cử Tổng thống Mỹ      0       news
1    2019-01-13  Bầu cử Tổng thống Mỹ      0       news
2    2019-01-20  Bầu cử Tổng thống Mỹ      0       news
3    2019-01-27  Bầu cử Tổng thống Mỹ      0       news
4    2019-02-03  Bầu cử Tổng thống Mỹ      0       news
...         ...                   ...    ...        ...
1035 2020-11-29         Du lịch Cô Tô      7     travel
1036 2020-12-06         Du lịch Cô Tô      0     travel
1037 2020-12-13         Du lịch Cô Tô     14     travel
1038 2020-12-20         Du lịch Cô Tô      0     travel
1039 2020-12-27         Du lịch Cô Tô      0     travel

[6760 rows x 4 columns]


In [20]:
def insertGoogleTrend2DB(data,fromDate,toDate):
    #df=getDataFromGoogleTrend('keytrends.xlsx',fromDate,toDate)
    df=data
    #Insert vao DB
    conn, cur = connect()
    #Xoa du lieu
    #truncate table
    cur.execute('delete from vn_trending where date between '+fromDate+' AND '+toDate+'')
    conn.commit()

    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in df.to_numpy()]

    # dataframe columns with Comma-separated
    cols = ','.join(list(df.columns))

    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES %%s" % ("vn_trending", cols)

    extras.execute_values(cur, sql, tpls)
    conn.commit()
    print("Data inserted using execute_values() successfully..")

In [21]:
insertGoogleTrend2DB('2019-01-01','2020-12-31')

Connecting to the PostgreSQL database...


UndefinedTable: relation "vn_trending" does not exist
LINE 1: delete from vn_trending where date between 2019-01-01 AND 20...
                    ^
